In [2]:
! conda --version

conda 4.7.5


## Beysian Inference
It is a method of inference where the probibility of a hypothesis is updated as new evidence becomes available
### Steps

** 1. Begin with a prior distribution processes : P(H)

** 2. collect data to obtain the observed distribution P(E) as evidence

** 3. calculate the likelihood of our observation given the hypothesis distribution (you can say we want to know how likely we can generate these observations data given aour hypothesis
distribution P(E|H))   

** 4. obtain the posterior or simple update the prior which is the correct distribution for another unseen data) P(H|E)

### bayes rule
$$
P(H|E) = \frac{P(E|H) \times P(H)}{p(E)} 
$$

## Maximum a posterior estimation

**note : the likelihood of a given sample $x$ is calculated as $p(x|\mu,\sigma) = \frac{1}{\sqrt{(2\pi \sigma^{2})}} \exp(-\frac{(x-\mu)^{2}}{2\sigma^{2}}) $

**example: we have two class of male and female and their height---> we can compute each class mean and std and then we can create our prior 

**now if given a new sample we can predict its class therefore we can do it for unseen samples

**The problem is as 

$$
Class = \armax_{c \in {c_{male},c_{female} } } P(c|x)
$$

therefor using bayes Rule and theank to likelihood that we can compute $p(x|c)$ we can reformulate and write general expression

$$
\max_{c\in C} \frac{p(x|c)\times p(c)}{p(x)}
$$

However we don't have $p(x)$ because we haven't seen all the x yet and there are a lots of unseen $x$ yet, but if you see the optimization it is independent from x, or better to say 
the probability distribution of data is independent from our classess because the data does not know and care about being male or female so we simply can forget about that in our optimization pronblem

*So
$
\max_{c\in C} p(x|c)\times p(c)
$